# Vertex Pipeline with KFP (Kube Flow Pipelines)

In [3]:
!pip install google-cloud-aiplatform==1.50.0 kfp==2.7.0

  Using cached google_cloud_aiplatform-1.50.0-py2.py3-none-any.whl.metadata (30 kB)
  Using cached protobuf-4.25.3-cp37-abi3-manylinux2014_x86_64.whl.metadata (541 bytes)
Using cached google_cloud_aiplatform-1.50.0-py2.py3-none-any.whl (4.9 MB)
Using cached protobuf-4.25.3-cp37-abi3-manylinux2014_x86_64.whl (294 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: google-cloud-aiplatform
    Found existing installation: google-cloud-aiplatform 1.48.0
    Uninstalling google-cloud-aiplatform-1.48.0:
      Successfully uninstalled google-cloud-aiplatform-1.48.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires protobuf<4,>3.12.2, but you have protobuf 4.25.3 which is incompatible.
google-api-python-cli

In [4]:
!pip install google-cloud-pipeline-components==2.14.0

In [5]:
YOUR_NAME="maha" # TO-DO: store your name in this var

In [6]:
PROJECT_ID='mlops-421501'
BUCKET=f"gs://{PROJECT_ID}-{YOUR_NAME}-bucket"

In [8]:
import kfp

from kfp.v2 import compiler, dsl
from kfp.v2.dsl import component, pipeline, Artifact, ClassificationMetrics, Input, Output, Model, Metrics

from google.cloud import aiplatform
import google_cloud_pipeline_components
from typing import NamedTuple

In [10]:
PATH=%env PATH
%env PATH={PATH}:/home/jupyter/.local/bin
REGION="us-central1"

PIPELINE_ROOT = f"{BUCKET}/pipeline_root/"
PIPELINE_ROOT

env: PATH=/usr/local/cuda/bin:/opt/conda/bin:/opt/conda/condabin:/usr/local/bin:/usr/bin:/bin:/usr/local/games:/usr/games:/home/jupyter/.local/bin:/home/jupyter/.local/bin


'gs://mlops-421501-maha-bucket/pipeline_root/'

In [15]:
@component(base_image="python:3.9")
def product_name(text: str) -> str:
    return text

In [16]:
@component(base_image="python:3.8", packages_to_install=["emoji"])
def emoji(
    text: str,
) -> NamedTuple(
    "Outputs",
    [
        ("emoji_text", str),  # Return parameters
        ("emoji", str),
    ],
):
    import emoji

    emoji_text = text
    emoji_str = emoji.emojize(':' + emoji_text + ':', language='alias')
    print("output one: {}; output_two: {}".format(emoji_text, emoji_str))
    return (emoji_text, emoji_str)

In [17]:
@component
def build_sentence(
    product: str,
    emoji: str,
    emojitext: str
) -> str:
    print("We completed the pipeline, hooray!")
    end_str = product + " is "
    if len(emoji) > 0:
        end_str += emoji
    else:
        end_str += emojitext
    return(end_str)

In [20]:
@pipeline(
    name="hello-world",
    description="An intro pipeline",
    pipeline_root=PIPELINE_ROOT,
)

# You can change the `text` and `emoji_str` parameters here to update the pipeline output
def intro_pipeline(
    text: str = "Vertex Pipelines", 
    emoji_str: str = "sparkles"
):
    product_task = product_name(text=text)
    emoji_task = emoji(text=emoji_str)
    consumer_task = build_sentence(
        product=product_task.output,
        emoji=emoji_task.outputs["emoji"],
        emojitext=emoji_task.outputs["emoji_text"],
    )

In [21]:
compiler.Compiler().compile(
    pipeline_func=intro_pipeline, package_path="intro_pipeline_job.json"
)

In [27]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

In [28]:
job = aiplatform.PipelineJob(
    display_name=f"{YOUR_NAME}-hello-world-pipeline",
    template_path="intro_pipeline_job.json",
    job_id="hello-world-pipeline-{0}".format(TIMESTAMP),
    enable_caching=True
)

In [29]:
job.submit()

Creating PipelineJob
PipelineJob created. Resource name: projects/578837671090/locations/us-central1/pipelineJobs/hello-world-pipeline-20240507062812
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/578837671090/locations/us-central1/pipelineJobs/hello-world-pipeline-20240507062812')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/hello-world-pipeline-20240507062812?project=578837671090
